In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

base_models = [
    ('rf', RandomForestClassifier()),
    ('xgb', XGBClassifier(eval_metric='logloss')),
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression()
)

stack_model.fit(X_train, y_train)
y_pred = stack_model.predict(X_test)


catboost

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1. Данные
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Стандартизация
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Базовые модели
cat = CatBoostClassifier(verbose=0, random_state=42)
rf = RandomForestClassifier(random_state=42)

# 4. OOF предсказания
cat_oof = cross_val_predict(cat, X_train_scaled, y_train, cv=5, method='predict_proba')[:, 1]
rf_oof = cross_val_predict(rf, X_train_scaled, y_train, cv=5, method='predict_proba')[:, 1]

# 5. Собираем входы мета-модели: OOF + признаки
meta_X_train = np.column_stack([cat_oof, rf_oof, X_train_scaled])

# 6. Обучим мета-модель
meta_model = CatBoostClassifier(verbose=0, random_state=42)
meta_model.fit(meta_X_train, y_train)

# 7. Тестовые предсказания от базовых
cat.fit(X_train_scaled, y_train)
rf.fit(X_train_scaled, y_train)
cat_test = cat.predict_proba(X_test_scaled)[:, 1]
rf_test = rf.predict_proba(X_test_scaled)[:, 1]

# 8. Финальный input: предсказания + X_test
meta_X_test = np.column_stack([cat_test, rf_test, X_test_scaled])
final_pred = meta_model.predict(meta_X_test)

# 9. Оценка
acc = accuracy_score(y_test, final_pred)
print(f"📊 Accuracy мета-модели CatBoost с passthrough=True: {acc:.4f}")


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score

# Загружаем данные
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Базовые модели
base_models = [
    ('cat', CatBoostClassifier(verbose=0, random_state=42)),
    ('rf', RandomForestClassifier(random_state=42))
]

# Финальная модель
final_model = LogisticRegression()

# Стэкинг
stacked = StackingClassifier(
    estimators=base_models,
    final_estimator=final_model,
    passthrough=False,
    cv=5
)

# Обучение и оценка
stacked.fit(X_train, y_train)
y_pred = stacked.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"🎯 Accuracy стэкинга (CatBoost + RF → LogisticRegression): {accuracy:.4f}")
